Code for scrapping

In [1]:
import requests
from bs4 import BeautifulSoup


def get_html_from_url(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        soup = BeautifulSoup(response.content, 'html.parser')
        return soup.prettify()
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
    except requests.exceptions.RequestException as req_err:
        print(f"Request exception occurred: {req_err}")
    except Exception as err:
        print(f"An error occurred: {err}")
    return None

In [2]:
# url = 'https://drt.etribunals.gov.in/drtlive/Misdetailreport.php?no=MDkwMTIwMDA5MTgyMDIzL2x1Y2tub3c='
url = "https://drt.etribunals.gov.in/drtlive/Misdetailreport.php?no=MDkwMTIwMDAwODUyMDIyL2x1Y2tub3c="
html_ = get_html_from_url(url)
# print(html_soup)
with open('code.html', '+w') as f:
    f.write(str(html_))
    f.close()

soup = BeautifulSoup(html_, 'html.parser')
# print(soup.prettify())
# print(soup.text)

In [3]:
import re
from unicodedata import normalize


def normalize_text(raw_text):
    return re.sub("\s\s+", " ", normalize("NFKD", raw_text.strip()))

In [4]:
def splitMystirng(text):
    lSting = text.split('\n')
    tempL = []
    for i in lSting:
        tempS = i.strip()
        if tempS != "":
            tempL.append(tempS)
    return tempL

In [5]:
table = soup.find(class_="table table-bordered table-extra-condensed")
table2RawString = normalize_text(table.text)
print(table2RawString)

# for case proceeding detail
tempTable = soup.find_all(class_="table table-bordered")
for i in tempTable:
    if i.find_all("tr") and normalize_text(i.find_all("tr")[0].text) == "CASE PROCEEDING DETAILS":
        caseProceedingTable = i
for i in caseProceedingTable.find_all('td'):
    if normalize_text(i.text) == "RC/TRC CASE CURRENT STATUS":
        break
    print(normalize_text(i.text))

CASE STATUS Diary no/Year 85/2022 Case Type/Case No/Year Original Application/50/2022 Date of Filing. 13/01/2022 Case Status. Pending CASE LISTING DETAILS (Tentative) In the Court of PO Court No. 1 Next Listing Date 10/08/2023 Next Listing Purpose APPEARANCE PETITIONER/APPLICANT DETAIL Petitioner Name -STATE BANK OF INDIA Petitioner/Applicant Address: SARB II 18 4 4TH FLOOR SBI HOUSE ARYA SAMAJ ROAD KAROL BAGH NEW DELHI Additional Party: Advocate Name: PRASHANT KUMAR SRIVASTAVAAdditional Advocate: RESPONDENTS/DEFENDENT DETAILS Respondent Name -VIVEK UPRAITY Respondent/Defendent Address: Additional Party: RUCHI TOMAR ,ANSAL URBAN CONDOMINIUMS , Advocate Name - Additional Advocate: PROPERTY DETAILS Property Type Detail Of Property IMMOVABLE FLAT NO ,02/10 D 02 ,ANSAL AQUAPOLI GHAZIABAD UP , CASE PROCEEDING DETAILS Court Name Causelist Date Purpose Registrar 10/02/2023 SERVICE STAGE PO 07/02/2023 APPEARANCE PO 20/10/2022 APPEARANCE PO 14/07/2022 APPEARANCE Registrar 22/03/2022 SERVICE STA

In [6]:
tempTable = soup.find_all(class_="table table-striped")
for i in tempTable:
    if i.find_all("tr") and normalize_text(i.find_all("tr")[0].text) == "PROPERTY DETAILS":
        caseProceedingTable = i
for i in caseProceedingTable.find_all('td'):
    print(normalize_text(i.text))

PROPERTY DETAILS
Property Type
Detail Of Property
IMMOVABLE
FLAT NO ,02/10 D 02 ,ANSAL AQUAPOLI GHAZIABAD UP ,


In [7]:
import simplematch
boundry = [
    "CASE STATUS {caseStatus} ",
    "CASE LISTING DETAILS (Tentative) {caseDetail} ",
    "PETITIONER/APPLICANT DETAIL {petDetail}",
    "RESPONDENTS/DEFENDENT DETAILS {resDetail}",
    "PROPERTY DETAILS {propertyDetailList}",
    "CASE PROCEEDING DETAILS {caseProceedingDetaillist}",
    "RC/TRC CASE CURRENT STATUS {rcTcsCaseCurrentStatus}",
    "RC CASE PROCEEDING DETAILS {rcCaseProceedingDetailList}",
]


def add_string_list(string_list):
    result = ""
    for string in string_list:
        result += string
    return result


dataByBoundry = simplematch.match(add_string_list(boundry), table2RawString)

# print(add_string_list(TABLE_CASE_STATUS_KEYS))
dataByBoundry

{'caseStatus': 'Diary no/Year 85/2022 Case Type/Case No/Year Original Application/50/2022 Date of Filing. 13/01/2022 Case Status. Pending',
 'caseDetail': 'In the Court of PO Court No. 1 Next Listing Date 10/08/2023 Next Listing Purpose APPEARANCE',
 'petDetail': 'Petitioner Name -STATE BANK OF INDIA Petitioner/Applicant Address: SARB II 18 4 4TH FLOOR SBI HOUSE ARYA SAMAJ ROAD KAROL BAGH NEW DELHI Additional Party: Advocate Name: PRASHANT KUMAR SRIVASTAVAAdditional Advocate: ',
 'resDetail': 'Respondent Name -VIVEK UPRAITY Respondent/Defendent Address: Additional Party: RUCHI TOMAR ,ANSAL URBAN CONDOMINIUMS , Advocate Name - Additional Advocate: ',
 'propertyDetailList': 'Property Type Detail Of Property IMMOVABLE FLAT NO ,02/10 D 02 ,ANSAL AQUAPOLI GHAZIABAD UP , ',
 'caseProceedingDetaillist': 'Court Name Causelist Date Purpose Registrar 10/02/2023 SERVICE STAGE PO 07/02/2023 APPEARANCE PO 20/10/2022 APPEARANCE PO 14/07/2022 APPEARANCE Registrar 22/03/2022 SERVICE STAGE PO 15/03/202

In [8]:
caseStatusKeys = [
    "Diary no/Year {diary_number} ",
    "Case Type/Case No/Year {case_type_number_year} ",
    "Date of Filing. {filing_date} ",
    "Case Status. {case_status} ",
    "Main Case -Diary No. {main_case_dairy_num} ",
    "Main Case - Case No. {main_case_num}",
]

caseDetailkeys = [
    "In the Court of {court_of} ",
    "Court No. {court_num} ",
    "Next Listing Date {next_listing_date} ",
    "Next Listing Purpose {next_listing_purpose}"
]

petDetailKeys = [
    "Petitioner Name -{pname} "
    "Petitioner/Applicant Address:{pet_address} ",
    "Additional Party: {additional_pet_party} ",
    "Advocate Name: {pet_adv}",
    "Additional Advocate: {additional_pet_adv}"
]

resDetailKeys = ["Respondent Name -{rname} ",
                 "Respondent/Defendent Address:{res_address} ",
                 "Additional Party: {add} ",
                 "Advocate Name -{respondent_advocates} ",
                 "Additional Advocate: {additional_res_adv}"
                 ]
rcTcsCseCurrentStatuskey = [
    "Court Name: -{courtName} ",
    "Next Listing Date: {nextListingDate} ",
    "Next Listing Purpose: {nextListingPurpose}"
]


def detailstrToList(keyStr, keyDict, detListStr):
    keyPattern = keyStr[0]
    for p in keyStr[1:]:
        keyPattern = keyPattern + " " + p
    keyPattern = keyPattern+" {data}"
    print(keyPattern)
    print(dataByBoundry['caseProceedingDetaillist'])
    temp_i = 0
    temp_l = simplematch.match(keyPattern, detListStr)['data'].split()
    output = []

    col_no = len(keyStr)
    temp_i = 0
    tempDict = {}
    for v in temp_l:
        if temp_i < col_no:
            tempDict[keyDict[temp_i]] = v
            temp_i = temp_i+1
        else:
            output.append(tempDict)
            temp_i = 0
            tempDict = {}
    return output


keysforcaseProceedingDetaillist = [
    "Court Name", "Causelist Date", "Purpose"]
keysfordictcaseProceedingDetaillist = [
    "court_name", "couselist_date", "purpose"
]


detailstrToList(keysforcaseProceedingDetaillist,
                keysfordictcaseProceedingDetaillist, dataByBoundry['caseProceedingDetaillist'])

Court Name Causelist Date Purpose {data}
Court Name Causelist Date Purpose Registrar 10/02/2023 SERVICE STAGE PO 07/02/2023 APPEARANCE PO 20/10/2022 APPEARANCE PO 14/07/2022 APPEARANCE Registrar 22/03/2022 SERVICE STAGE PO 15/03/2022 APPEARANCE Registrar 04/03/2022 FRESH MATTERS 


[{'court_name': 'Registrar',
  'couselist_date': '10/02/2023',
  'purpose': 'SERVICE'},
 {'court_name': 'PO', 'couselist_date': '07/02/2023', 'purpose': 'APPEARANCE'},
 {'court_name': '20/10/2022', 'couselist_date': 'APPEARANCE', 'purpose': 'PO'},
 {'court_name': 'APPEARANCE',
  'couselist_date': 'Registrar',
  'purpose': '22/03/2022'},
 {'court_name': 'STAGE', 'couselist_date': 'PO', 'purpose': '15/03/2022'},
 {'court_name': 'Registrar',
  'couselist_date': '04/03/2022',
  'purpose': 'FRESH'}]

In [ ]:
[{'court_name': 'PO', 'couselist_date': '08/09/2022', 'purpose': 'JUDGMENT'}, {'court_name': '21/07/2022', 'couselist_date': 'APPEARANCE', 'purpose': 'PO'},
    {'court_name': 'APPEARANCE', 'couselist_date': 'PO', 'purpose': '07/04/2022'}, {'court_name': 'PO', 'couselist_date': '16/03/2022', 'purpose': 'APPEARANCE'}]
[{'court_name': 'PO', 'couselist_date': '08/09/2022', 'purpose': 'JUDGMENT'}, {'court_name': '21/07/2022', 'couselist_date': 'APPEARANCE', 'purpose': 'PO'},
    {'court_name': 'APPEARANCE', 'couselist_date': 'PO', 'purpose': '07/04/2022'}, {'court_name': 'PO', 'couselist_date': '16/03/2022', 'purpose': 'APPEARANCE'}]